<a href="https://colab.research.google.com/github/ZhanglinX233/mc_server/blob/main/colab_webui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

环境配置 environment

In [ ]:
!pip install -q condacolab
# Setting up condacolab and installing packages
import condacolab
condacolab.install_from_url("https://repo.anaconda.com/miniconda/Miniconda3-py39_23.11.0-2-Linux-x86_64.sh")
%cd -q /content
!git clone https://github.com/RVC-Boss/GPT-SoVITS
!conda install -y -q -c pytorch -c nvidia cudatoolkit
%cd -q /content/GPT-SoVITS
!conda install -y -q -c conda-forge gcc gxx ffmpeg cmake -c pytorch -c nvidia
!/usr/local/bin/pip install -r requirements.txt

In [ ]:
# @title Download pretrained models 下载预训练模型
!mkdir -p /content/GPT-SoVITS/GPT_SoVITS/pretrained_models
!mkdir -p /content/GPT-SoVITS/tools/damo_asr/models
!mkdir -p /content/GPT-SoVITS/tools/uvr5
%cd /content/GPT-SoVITS/GPT_SoVITS/pretrained_models
!git clone https://huggingface.co/lj1995/GPT-SoVITS
%cd /content/GPT-SoVITS/tools/damo_asr/models
!git clone https://www.modelscope.cn/damo/speech_paraformer-large_asr_nat-zh-cn-16k-common-vocab8404-pytorch.git
!git clone https://www.modelscope.cn/damo/speech_fsmn_vad_zh-cn-16k-common-pytorch.git
!git clone https://www.modelscope.cn/damo/punc_ct-transformer_zh-cn-common-vocab272727-pytorch.git
# @title UVR5 pretrains 安装uvr5模型
%cd /content/GPT-SoVITS/tools/uvr5
%rm -r uvr5_weights
!git clone https://huggingface.co/Delik/uvr5_weights
!git config core.sparseCheckout true
!mv /content/GPT-SoVITS/GPT_SoVITS/pretrained_models/GPT-SoVITS/* /content/GPT-SoVITS/GPT_SoVITS/pretrained_models/

In [ ]:
# @title launch WebUI 启动WebUI
!/usr/local/bin/pip install ipykernel
!sed -i '10s/False/True/' /content/GPT-SoVITS/config.py
%cd /content/GPT-SoVITS/
!/usr/local/bin/python  webui.py

In [8]:
!/usr/local/bin/pip install parsel

In [ ]:
from json import dump
import requests
from parsel import Selector
from pymongo import MongoClient
from bson.objectid import ObjectId
import os

cookies = {
    'gamecenter_wiki__session': '8er0alovkgjsq6u7ssnf80vut1q66s1c',
    'gamecenter_wiki_UserID': '7078378',
    'gamecenter_wiki_UserName': '514671750',
    'gamecenter_wiki_UserGroups': 'bilibili',
    'DedeUserID': '514671750',
    'DedeUserID__ckMd5': '9b4aa2e41166602b',
    'SESSDATA': 'f8f68a41%2C1751077808%2C9882f%2Ac2CjCvhgwn1Gb34vUxwFWuQH7tbeAGyDQBjVcaO8fka7J4Td78kK9CkiPiwu7rhe5e7kESVklkMVNMZGg3aDl6bDFFVVhaU3RQdHVyNGFOeXBsS0JsbHJKMjB2dmJreHVPc3c2bi1hNmg4NzJ0VEp4eTBFbTlXZ3FZU1NxaDY4VXdVaFhZQ2V3QWN3IIEC',
    'bili_jct': 'd2ca971ca10a2403ab634c01f95ae328',
    'sid': '66z88uox',
    'b_lsid': '3A9A3936_1943FC37C14',
    'buvid4': '73735C57-D580-5477-FE3D-1D1673B3661090519-024101710-BtAsIXQgcHCLJyLXMiqnDg%3D%3D',
    'b_nut': '1736236563',
    'buvid3': 'BA9B2FC8-0BD1-B300-6E80-DCC71B2345D363622infoc',
    'buvid_fp': '722f57d252ea7e2312d3b8085aecb27a',
    'Hm_lvt_cb50e488eca598646f26b3bf09b83ada': '1736236564',
    'HMACCOUNT': 'A6B4B6C2799017B6',
    'bsource': 'search_google',
    'Hm_lpvt_cb50e488eca598646f26b3bf09b83ada': '1736236582',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'zh-CN,zh;q=0.9,en-US;q=0.8,en;q=0.7',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    # 'Cookie': 'gamecenter_wiki__session=8er0alovkgjsq6u7ssnf80vut1q66s1c; gamecenter_wiki_UserID=7078378; gamecenter_wiki_UserName=514671750; gamecenter_wiki_UserGroups=bilibili; DedeUserID=514671750; DedeUserID__ckMd5=9b4aa2e41166602b; SESSDATA=f8f68a41%2C1751077808%2C9882f%2Ac2CjCvhgwn1Gb34vUxwFWuQH7tbeAGyDQBjVcaO8fka7J4Td78kK9CkiPiwu7rhe5e7kESVklkMVNMZGg3aDl6bDFFVVhaU3RQdHVyNGFOeXBsS0JsbHJKMjB2dmJreHVPc3c2bi1hNmg4NzJ0VEp4eTBFbTlXZ3FZU1NxaDY4VXdVaFhZQ2V3QWN3IIEC; bili_jct=d2ca971ca10a2403ab634c01f95ae328; sid=66z88uox; b_lsid=3A9A3936_1943FC37C14; buvid4=73735C57-D580-5477-FE3D-1D1673B3661090519-024101710-BtAsIXQgcHCLJyLXMiqnDg%3D%3D; b_nut=1736236563; buvid3=BA9B2FC8-0BD1-B300-6E80-DCC71B2345D363622infoc; buvid_fp=722f57d252ea7e2312d3b8085aecb27a; Hm_lvt_cb50e488eca598646f26b3bf09b83ada=1736236564; HMACCOUNT=A6B4B6C2799017B6; bsource=search_google; Hm_lpvt_cb50e488eca598646f26b3bf09b83ada=1736236582',
    'Pragma': 'no-cache',
    'Referer': 'https://www.google.com/',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'cross-site',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

def get_url():
    params = {
        'biliLogin': '1',
    }

    response = requests.get(
        'https://wiki.biligame.com/ys/%E5%88%86%E7%B1%BB:%E8%A7%92%E8%89%B2%E8%AF%AD%E9%9F%B3',
        params=params,
        cookies=cookies,
        headers=headers,
    )

    if response.status_code == 200:
        ele = Selector(response.text)
        names = ele.css('div.mw-category ul>li>a::attr(title)').getall()
        urls = ele.css('div.mw-category ul>li>a::attr(href)').getall()
        datas = [{'name': name, 'url': url, 'status': None} for name, url in zip(names, urls)]

        for name, url in datas:
            print(url)
            response = requests.get(
                url='https://wiki.biligame.com' + url,
                cookies=cookies,
                headers=headers,
            )
            data_dict = {'zh': [], 'jp': [], 'en': [], 'kr': [], 'status': 'yes'}
            if response.status_code == 200:
                print('请求成功')
                ele = Selector(response.text)
                tbodys = ele.css('div.visible-xs>table.wikitable>tbody')
                for body in tbodys[1:]:
                    lines = body.css('tr:last-child div::text').getall()
                    cvs = body.css('tr div.bikit-audio::attr(data-src)').getall()
                    langs = ['zh', 'jp', 'en', 'kr']
                    for line, cv, lang in zip(lines, cvs, langs):
                        data_dict[lang].append({'line': line, 'cv': cv})
                for l in ['zh', 'jp', 'en', 'kr']:
                    if not os.path.exists(f'{name}'):
                        os.mkdir(f'{name}')
                    with open(f'{name}/{l}.json', 'w', encoding='utf-8')as file:
                        dump(file, data_dict[l])


if __name__ == '__main__':
    get_url()
